In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import pickle
import tensorflow as tf

In [2]:
image_width = 320
image_height = 240

In [3]:
def label_file(x):
    return 'Emotion_labels/Emotion/'+x[0]+'/'+x[1]+'/'+x[2]+'_emotion.txt'
def image_file(x):
    return 'extended-cohn-kanade-images/cohn-kanade-images/'+x[0]+'/'+x[1]+'/'+x[2]+'.png'

def label_read(filename):
    file = open(filename, 'r')
    out = []
    for line in file:
        out.append(line.strip())
    return np.float16(out[0])

In [4]:
labels_filename = 'labels.pickle'
images_filename = 'images1.pickle'

In [5]:
if not glob.os.path.exists(labels_filename) or not glob.os.path.exists(images_filename):
    file_list = [(file[23:27], file[28:31], file[32:49]) for file in glob.glob('Emotion_labels/Emotion/*/*/*.txt')]
    data_files = [(label_file(file), image_file(file)) for file in file_list]
    images = np.array([cv2.imread(file[1]) for file in data_files])
    images = np.array([cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) for image in images])/255
    images = np.array([cv2.resize(image, (image_width, image_height)) for image in images]).astype(np.float16)
    labels = np.array([label_read(file[0]) for file in data_files]) -1
    random = np.arange(0,len(labels)).astype(np.uint64)
    np.random.shuffle(random)
    images = images[random]
    labels = labels[random]
    pickle.dump(images, open(images_filename, 'wb'))
    pickle.dump(labels, open(labels_filename, 'wb'))
else:
    print('pickle file is exist')

pickle file is exist


In [6]:
images = pickle.load(open(images_filename, "rb" ) )
labels = pickle.load(open(labels_filename, "rb" ) )

In [7]:
images = images.reshape(images.shape[0], image_width, image_height, 1)

In [8]:
train_images = images[:280]
train_labels = labels[:280]
test_images = images[281:]
test_labels = labels[281:]

In [9]:
def plot_hist(hist):
    plt.figure(figsize=(16,4))
    plt.subplot(1,2,1)
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.subplot(1,2,2)
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    random_acc = np.max(np.bincount(test_labels.astype(np.int16))/len(test_labels))
    plt.hlines(y=random_acc, xmin=0, xmax=len(hist.history['acc']), colors='red')

def plot_hists(hists):
    for i in range(len(hists)):
        if not hists[i] == None:
            plt.figure(figsize=(16,4*len(hists)))
            plt.subplot(len(hists),2,2*i+1)
            plt.plot(hists[i].history['loss'])
            plt.plot(hists[i].history['val_loss'])
            plt.subplot(len(hists),2,2*i+2)
            plt.plot(hists[i].history['acc'])
            plt.plot(hists[i].history['val_acc'])
            random_acc = np.max(np.bincount(test_labels.astype(np.int16))/len(test_labels))
            plt.hlines(y=random_acc, xmin=0, xmax=len(hists[i].history['acc']), colors='red')

In [13]:
def create_model(dense):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(train_images.shape[1:])),
        tf.keras.layers.Conv2D(256, strides=(2,2), kernel_size=(3,3)),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Conv2D(64, strides=(1,1), kernel_size=(3,3)),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(32, strides=(2,2), kernel_size=(3,3)),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Conv2D(32, strides=(1,1), kernel_size=(3,3)),
        tf.keras.layers.Conv2D(16, strides=(1,1), kernel_size=(3,3)),
        tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(512, activation=tf.nn.relu),
#         tf.keras.layers.Dropout(0.4),
#         tf.keras.layers.Dense(256, activation=tf.nn.relu),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Dense(128, activation=tf.nn.relu),
        tf.keras.layers.Dense(7, activation=tf.nn.softmax)
    ])
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    print(model.summary())
    return model

def fit(model, epochs=50):
    return model.fit(train_images, train_labels, epochs=epochs, validation_data=(test_images, test_labels))

In [14]:
model = create_model(80)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 159, 119, 256)     2560      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 79, 59, 256)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 79, 59, 256)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 77, 57, 64)        147520    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 38, 28, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 38, 28, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 18, 13, 32)        18464     
__________

In [15]:
hist = fit(model, 100)

Train on 280 samples, validate on 46 samples
Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[32,256,79,59] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_1/TFOptimizer/gradients/conv2d_6/Conv2D_grad/Conv2DBackpropInput = Conv2DBackpropInput[T=DT_FLOAT, _class=["loc:@training_1/TFOptimizer/gradients/dropout_5/cond/Merge_grad/cond_grad"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/TFOptimizer/gradients/conv2d_6/Conv2D_grad/ShapeN, conv2d_6/Conv2D/ReadVariableOp, training_1/TFOptimizer/gradients/max_pooling2d_4/MaxPool_grad/MaxPoolGrad, ^training_1/TFOptimizer/gradients/conv2d_6/BiasAdd_grad/BiasAddGrad)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
plot_hist(hist)

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
np.max(np.bincount(test_labels.astype(np.int16))/len(test_labels))

In [ ]:
def pred(image):
    img_temp = cv2.resize(image, (image_height, image_width))
    img_temp = img_temp/255
    img_temp = cv2.cvtColor(img_temp, cv2.COLOR_RGB2GRAY)
    img_temp = img_temp.reshape(img_temp.shape[0],img_temp.shape[1],1)
    return model.predict(np.array([img_temp]))
    
def pred_text(prediction):
    text = []
    for i in range(len(prediction)):
        text.append("%s : %.2f%%\n"%(i, prediction[i]*100))
    return text

In [ ]:
# cap = cv2.VideoCapture(0)

# while(True):
#     # Capture frame-by-frame
#     ret, frame = cap.read()

#     # Our operations on the frame come here
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     font = cv2.FONT_HERSHEY_SIMPLEX
#     prediction = pred(frame.astype(np.float32))[0]
#     predicted_texts = pred_text(prediction)
#     for i in range(len(predicted_texts)):
#         cv2.putText(frame,predicted_texts[i],(0,20+i*30), font, 0.5,(np.round(255-255*prediction[i]),np.round(255*prediction[i]),255-max(np.round(255-255*prediction[i]),np.round(255*prediction[i]))),2,cv2.LINE_AA)
#     # Display the resulting frame
#     cv2.imshow('frame',frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # When everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
predictions = model.predict(test_images)

In [ ]:
np.argmax(predictions, axis=1)